In [2]:
!pip3 install transforms
!pip3 install sgmllib3k
#!unrar x dataset_xy.rar
!unzip output.zip

Archive:  output.zip
   creating: output/
  inflating: output/resnet50_lr1e-05_epochs50_batch16.pth  
  inflating: output/resnet50_lr1e-05_epochs50_batch16_trt_fp32.pth  
  inflating: output/resnet18_lr1e-05_epochs50_batch16.pth  
  inflating: output/resnet34_lr1e-05_epochs50_batch16.pth  
  inflating: output/resnet18_lr1e-05_epochs50_batch16_trt_fp8.pth  
  inflating: output/resnet50_lr1e-05_epochs50_batch16_trt_fp16.pth  
  inflating: output/resnet18_lr1e-05_epochs50_batch16_trt_fp16.pth  
   creating: output/.ipynb_checkpoints/
  inflating: output/resnet34_lr1e-05_epochs50_batch16_trt_fp8.pth  
  inflating: output/resnet50_lr1e-05_epochs50_batch16_trt_fp8.pth  
  inflating: output/resnet34_lr1e-05_epochs50_batch16_trt_fp16.pth  
  inflating: output/resnet34_lr1e-05_epochs50_batch16_trt_fp32.pth  
  inflating: output/resnet18_lr1e-05_epochs50_batch16_trt_fp32.pth  


In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn.functional as F
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np
import torch.nn as nn
import ipywidgets as widgets
from IPython.display import display
import csv
import ipywidgets as widgets
from jetbot import Robot
import numpy as np
from datetime import datetime
from jetbot import Camera, bgr8_to_jpeg
import torch
import cv2
import traitlets
import numpy as np
from jetbot import Camera, bgr8_to_jpeg
from torchvision import models, transforms
from ipywidgets import widgets
from IPython.display import display
import time
import torch
import cv2
import traitlets
import numpy as np
from jetbot import Camera, bgr8_to_jpeg
from torchvision import models, transforms
from ipywidgets import widgets
from IPython.display import display
import time
import ipywidgets

In [14]:
# 初始化相機
camera = Camera.instance(width=224, height=224,fps=5)

# 設定 widget 大小
widget_width = 224
widget_height = 224

# 建立顯示影像的 widget 和 x, y 控制滑桿
#image_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
#target_widget = widgets.Image(format='jpeg', width=widget_width, height=widget_height)
#x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
#y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

# 顯示 widget
#display(widgets.HBox([image_widget]))

# 啟動相機影像的即時處理和推論
time.sleep(1)  # 確保相機啟動完成
#traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
#traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=process_and_predict)


In [17]:
camera.stop()

In [3]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.02, description='steering kd')
steering_left_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias left')
steering_right_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias right')
steering_igain_slider = widgets.FloatSlider(
    min=0.0, max=1.0, step=0.01, value=0.00, description='Steering I Gain'
)
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_left_bias_slider,steering_right_bias_slider,steering_igain_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.1, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.02, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias left', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='steering bias right', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='Steering I Gain', max=1.0, step=0.01)

In [4]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, step=0.01 ,description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, orientation='vertical', description='speed')
display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0, step=0.01)

In [9]:
# 設定裝置 (若有 GPU 可用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 加載模型
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load("lane_detection_model.pth"))
model = model.to(device)
model.eval()  # 將模型設定為推論模式

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4691, 0.4032, 0.4579], [0.1740, 0.1485, 0.1688])
    ])

# 初始化機器人和滑桿
robot = Robot()
robot.motor_driver._pwm.setPWMFreq(100)
angle = 0.0
angle_last = 0.0
integral = 0.0  # 初始化積分項
left_motor_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, description='Left Motor')
right_motor_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, description='Right Motor')
angle_slider = widgets.FloatSlider(min=-np.pi, max=np.pi, step=0.01, description='Angle')
display(left_motor_slider, right_motor_slider, angle_slider)

# 初始化 CSV 文件，添加標題
with open('motor_log.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['timestamp', 'left_motor', 'right_motor', 'speed', 'angle'])

def execute(change):
    import time
    start_time = time.time()  # 開始計時
    global angle, angle_last, integral
    image = change['new']
    
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(input_tensor)
        x, y = output[0].cpu().numpy()
    
    x_slider.value = x
    y_slider.value = y
    print(x,y)
    speed_slider.value = speed_gain_slider.value
    
    # 計算角度，更新 PID 項
    angle = np.arctan2(x, y)
    angle_slider.value = angle  # 更新角度滑桿值

    # 更新積分項
    integral += angle
    
    # 計算 PID 控制值
    pid = (
        angle * steering_gain_slider.value  # P 項
        + (angle - angle_last) * steering_dgain_slider.value  # D 項
        + integral * steering_igain_slider.value  # I 項
    )
    
    angle_last = angle  # 更新上一個角度

    if angle < 0:
        steering_slider.value = pid + steering_left_bias_slider.value  # 如果 angle < 0，添加偏置
    else:
        steering_slider.value = pid + steering_right_bias_slider.value  # 如果 angle >= 0，減去偏置
    
    # 計算馬達值
    right_motor_value = -max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    left_motor_value = -max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    left_motor_slider.value = left_motor_value
    right_motor_slider.value = right_motor_value
    
    # 設置機器人馬達值
    robot.left_motor.value = left_motor_value
    robot.right_motor.value = right_motor_value
    
    end_time = time.time()  # 結束計時
    execution_time = end_time - start_time  # 計算間隔時間
    print(f'Execution time: {execution_time:.4f} seconds')

execute({'new': camera.value})


cuda


FloatSlider(value=0.0, description='Left Motor', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.0, description='Right Motor', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.0, description='Angle', max=3.141592653589793, min=-3.141592653589793, step=0.01)

-0.1504682 0.5497326
Execution time: 0.5595 seconds


In [15]:
camera.observe(execute, names='value')

0.122085206 0.6316299
Execution time: 0.5251 seconds
0.12429183 0.631281
Execution time: 0.2432 seconds
0.1194069 0.6257095
Execution time: 0.2564 seconds
0.121673174 0.63035285
Execution time: 0.2276 seconds
0.12557882 0.63187987
Execution time: 0.2278 seconds
0.12443396 0.6300683
Execution time: 0.2217 seconds
0.13028827 0.6265801
Execution time: 0.2666 seconds
0.1203733 0.62656224
Execution time: 0.2539 seconds
0.11708953 0.6307268
Execution time: 0.2280 seconds
0.121808656 0.6325695
Execution time: 0.2384 seconds
0.12224423 0.63465625
Execution time: 0.2165 seconds
0.11888268 0.6314834
Execution time: 0.2137 seconds
0.1173001 0.62953424
Execution time: 0.2193 seconds
0.123165734 0.63078356
Execution time: 0.2734 seconds
0.121271424 0.63423276
Execution time: 0.2481 seconds
0.12555107 0.62943584
Execution time: 0.2284 seconds
0.120726295 0.6350983
Execution time: 0.2284 seconds
0.12449431 0.62962013
Execution time: 0.2432 seconds
0.12978548 0.63072336
Execution time: 0.2380 seconds


In [16]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

-0.19013232 0.57830393
Execution time: 0.2332 seconds
-0.18839103 0.57769483
Execution time: 0.2478 seconds
